In [1]:
spark

In [2]:
spark.range(1, 10).show()

+---+
| id|
+---+
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



In [3]:
df = spark.read.option("header", "true") \
    .csv("gs://anshk-data/Data/1MB/customers.csv")

df.limit(5).show()

+-----------+----------+---------+-----------+-------+-----------------+---------+
|customer_id|      name|     city|      state|country|registration_date|is_active|
+-----------+----------+---------+-----------+-------+-----------------+---------+
|          0|Customer_0|     Pune|Maharashtra|  India|       2023-06-29|    False|
|          1|Customer_1|Bangalore| Tamil Nadu|  India|       2023-12-07|     True|
|          2|Customer_2|Hyderabad|    Gujarat|  India|       2023-10-27|     True|
|          3|Customer_3|Bangalore|  Karnataka|  India|       2023-10-17|    False|
|          4|Customer_4|Ahmedabad|  Karnataka|  India|       2023-03-14|    False|
+-----------+----------+---------+-----------+-------+-----------------+---------+



In [5]:
df.rdd.getNumPartitions()

1

In [7]:
df_rdd = df.rdd

In [8]:
header = df_rdd.first()

In [10]:
header

Row(customer_id='0', name='Customer_0', city='Pune', state='Maharashtra', country='India', registration_date='2023-06-29', is_active='False')

In [13]:
df_rdd.take(5)

[Row(customer_id='0', name='Customer_0', city='Pune', state='Maharashtra', country='India', registration_date='2023-06-29', is_active='False'),
 Row(customer_id='1', name='Customer_1', city='Bangalore', state='Tamil Nadu', country='India', registration_date='2023-12-07', is_active='True'),
 Row(customer_id='2', name='Customer_2', city='Hyderabad', state='Gujarat', country='India', registration_date='2023-10-27', is_active='True'),
 Row(customer_id='3', name='Customer_3', city='Bangalore', state='Karnataka', country='India', registration_date='2023-10-17', is_active='False'),
 Row(customer_id='4', name='Customer_4', city='Ahmedabad', state='Karnataka', country='India', registration_date='2023-03-14', is_active='False')]

In [14]:
df.take(1)

[Row(customer_id='0', name='Customer_0', city='Pune', state='Maharashtra', country='India', registration_date='2023-06-29', is_active='False')]

In [16]:
cust_per_city = df_rdd.map(lambda row:row[2]).countByValue()

In [17]:
cust_per_city

defaultdict(int,
            {'Pune': 2243,
             'Bangalore': 2211,
             'Hyderabad': 2242,
             'Ahmedabad': 2198,
             'Mumbai': 2142,
             'Delhi': 2200,
             'Chennai': 2194,
             'Kolkata': 2223})

In [18]:
from pyspark.sql.functions import col

In [19]:
custCity = (df.groupBy(col('city')).count())

In [21]:
custCity.show()

+---------+-----+
|     city|count|
+---------+-----+
|    Delhi| 2200|
|  Kolkata| 2223|
|Hyderabad| 2242|
|Bangalore| 2211|
|Ahmedabad| 2198|
|  Chennai| 2194|
|   Mumbai| 2142|
|     Pune| 2243|
+---------+-----+



In [22]:
df_rdd.take(1)

[Row(customer_id='0', name='Customer_0', city='Pune', state='Maharashtra', country='India', registration_date='2023-06-29', is_active='False')]

In [28]:
active_cust = df_rdd.filter(lambda row:row.is_active == 'True') \
                    .map(lambda row:row.state) \
                    .countByValue()

In [29]:
active_cust

defaultdict(int,
            {'Tamil Nadu': 1252,
             'Gujarat': 1332,
             'West Bengal': 1197,
             'Karnataka': 1276,
             'Telangana': 1226,
             'Delhi': 1222,
             'Maharashtra': 1230})

In [31]:
sc.parallelize(active_cust.items()) \
  .saveAsTextFile("/tmp/active_cities")


In [32]:
!hadoop fs -ls /tmp

Found 3 items
drwxr-xr-x   - root hadoop          0 2025-12-16 20:44 /tmp/active_cities
drwxrwxrwt   - hdfs hadoop          0 2025-12-11 19:47 /tmp/hadoop-yarn
drwx-wx-wx   - hive hadoop          0 2025-12-11 19:47 /tmp/hive


In [33]:
spark.stop()